In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

In [6]:
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/trainset.csv",encoding='ISO-8859-1')
valid_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/validationset.csv",encoding='ISO-8859-1')
test_df =pd.read_csv("/content/drive/MyDrive/PFA/DATASET/testset.csv",encoding='ISO-8859-1')

In [4]:
def extract_images(df,output):
  individual_captions = pd.DataFrame(columns=["Media","I-E","S-N","T-F","J-P"])

  for index,row in df.iterrows():
    for caption_index,caption in enumerate(eval(row['Medias'])):
      # Define the new row to be added
      new_row = {'Media': caption.strip("\n"),"I-E":df["I-E"][index],\
                 'S-N':df["S-N"][index],'T-F':df["T-F"][index],'J-P':df["J-P"][index]}
      # Use the loc method to add the new row to the DataFrame
      individual_captions.loc[len(individual_captions)] = new_row
  individual_captions.to_csv(output)
  return individual_captions
  

# Generate dataframes in not exist as csv already.

In [2]:
train_df_indiv = extract_images(train_df,"/content/drive/MyDrive/PFA/DATASET/indiv_img_trainset.csv")

NameError: ignored

In [7]:
valid_df_indiv = extract_images(valid_df,"/content/drive/MyDrive/PFA/DATASET/indiv_img_validationset.csv")

NameError: ignored

In [ ]:
test_df_indiv = extract_images(test_df,"/content/drive/MyDrive/PFA/DATASET/indiv_img_testset.csv")

# Read dataframes if already been generated previously

In [8]:
test_df_indiv = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/indiv_img_testset.csv")
valid_df_indiv = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/indiv_img_validationset.csv")
train_df_indiv = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/indiv_img_trainset.csv")

# Shuffle for more consistency

In [11]:
train_df_indiv = train_df_indiv.sample(frac = 1)
valid_df_indiv = valid_df_indiv.sample(frac = 1)

In [12]:
train_df_indiv

,Unnamed: 0,Media,I-E,S-N,T-F,J-P
14706,14706,/content/drive/MyDrive/PFA/DATASET/Data/christ...,0,0,0,1
10384,10384,/content/drive/MyDrive/PFA/DATASET/Data/jeremy...,1,1,0,0
2028,2028,/content/drive/MyDrive/PFA/DATASET/Data/leafgi...,0,0,0,0
14943,14943,/content/drive/MyDrive/PFA/DATASET/Data/captke...,1,1,0,0
11973,11973,/content/drive/MyDrive/PFA/DATASET/Data/glitze...,0,1,0,0
...,...,...,...,...,...,...
9165,9165,/content/drive/MyDrive/PFA/DATASET/Data/johand...,1,1,0,0
12164,12164,/content/drive/MyDrive/PFA/DATASET/Data/garden...,1,1,0,0
11513,11513,/content/drive/MyDrive/PFA/DATASET/Data/greatw...,1,1,0,0
3205,3205,/content/drive/MyDrive/PFA/DATASET/Data/_its_d...,0,1,0,1


In [16]:
def read_images_convert_to_x_y(dataframe,y_classification = "I-E"):
  images = []
  labels = []
  for index,row in dataframe.iterrows():
    image = cv2.imread(row['Media'])
    resized_image = cv2.resize(image,(244,244))
    images.append(resized_image)
    labels.append(row[y_classification])
  # Convert the frames to a numpy array and preprocess them
  images = np.array(images, dtype=np.float32) / 255.0
  images -= np.array([0.485, 0.456, 0.406])
  images /= np.array([0.229, 0.224, 0.225])
  return images, np.array(labels)
train_images, train_labels = read_images_convert_to_x_y(train_df_indiv)
valid_images, valid_labels = read_images_convert_to_x_y(valid_df_indiv)

error: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Define a simple CNN model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

history = model.fit(train_images, train_labels, epochs=5, batch_size=32, validation_data=(valid_images, valid_labels))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

classifier = Sequential()
classifier.add(Conv2D(32, kernel_size = 3, padding = 'same', input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = 2))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale= 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)                     

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set', target_size = (64, 64), batch_size = 32, class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set', target_size = (64, 64), batch_size = 32, class_mode = 'binary')              

classifier.fit(training_set, steps_per_epoch = 8000, epochs = 25, validation_data = test_set, validation_steps = 2000, verbose = 2)